In [5]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

ASEAN5_COUNTRIES = ['Indonesia', 'Malaysia', 'Filipina', 'Singapura', 'Thailand']
START_YEAR = 1995
END_YEAR = 2024


In [6]:
data_file = "Data/International Finance Dinda.xlsx"
xl_file = pd.ExcelFile(data_file)
print(f"Available sheets: {xl_file.sheet_names}")

all_sheets = []
for sheet_name in xl_file.sheet_names:
    sheet_df = pd.read_excel(xl_file, sheet_name=sheet_name)
    sheet_df['country'] = sheet_name
    all_sheets.append(sheet_df)
    print(f"Sheet '{sheet_name}': {sheet_df.shape}")

df = pd.concat(all_sheets, ignore_index=True)
print(f"\nMerged shape: {df.shape}")
print(f"Columns before drop: {list(df.columns)}")

cols_to_drop = ['Year', 'Indonesia', 'Malaysia', 'Philippines', 'Singapore', 'Thailand', 'Filipina', 'Singapura']
df = df.drop(columns=[col for col in cols_to_drop if col in df.columns], errors='ignore')
print(f"Columns after drop: {list(df.columns)}")
print(f"Final shape: {df.shape}")
df.head()


Available sheets: ['Indonesia', 'Malaysia', 'Filipina', 'Singapura', 'Thailand', 'Official Exchange Rate Local Cu', 'Corruption Perception Index', 'GDP Growth', 'Current Account', 'Inflation (Consumer Price Index', 'Total Reserve in Month Imports ']
Sheet 'Indonesia': (30, 8)
Sheet 'Malaysia': (30, 8)
Sheet 'Filipina': (30, 8)
Sheet 'Singapura': (30, 8)
Sheet 'Thailand': (30, 8)
Sheet 'Official Exchange Rate Local Cu': (30, 7)
Sheet 'Corruption Perception Index': (30, 7)
Sheet 'GDP Growth': (30, 7)
Sheet 'Current Account': (30, 7)
Sheet 'Inflation (Consumer Price Index': (30, 7)
Sheet 'Total Reserve in Month Imports ': (30, 7)

Merged shape: (330, 16)
Columns before drop: ['Tahun', 'Official Exchange Rate: Period Average: Local Currency to USD', 'Corruption Perception Index', 'GDP Growth(%)', 'Current account balance (% of GDP)', 'Inflation, consumer prices (annual %)', 'Total Reserves in Month Imports', 'country', 'Year', 'Indonesia', 'Malaysia', 'Philippines', 'Singapore', 'Thailand'

,Tahun,Official Exchange Rate: Period Average: Local Currency to USD,Corruption Perception Index,GDP Growth(%),Current account balance (% of GDP),"Inflation, consumer prices (annual %)",Total Reserves in Month Imports,country
0,1995.0,2243.6523,19.4,8.220007,-3.181584,9.420323,2.902138,Indonesia
1,1996.0,2327.3309,26.5,7.818187,-3.370282,7.973281,3.494959,Indonesia
2,1997.0,2879.7746,27.2,4.699879,-2.266059,6.226142,2.954808,Indonesia
3,1998.0,10193.9768,20.0,-13.126700,4.292464,58.451044,5.233226,Indonesia
4,1999.0,7854.9861,17.0,0.791126,4.130602,20.477831,6.092295,Indonesia


In [7]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330 entries, 0 to 329
Data columns (total 8 columns):
 #   Column                                                         Non-Null Count  Dtype  
---  ------                                                         --------------  -----  
 0   Tahun                                                          300 non-null    float64
 1   Official Exchange Rate: Period Average: Local Currency to USD  150 non-null    float64
 2   Corruption Perception Index                                    150 non-null    float64
 3   GDP Growth(%)                                                  150 non-null    float64
 4   Current account balance (% of GDP)                             150 non-null    float64
 5   Inflation, consumer prices (annual %)                          150 non-null    float64
 6   Total Reserves in Month Imports                                150 non-null    float64
 7   country                                                        

In [8]:
missing = df.isnull().sum()
print("Missing values per kolom:")
print(missing[missing > 0] if missing.sum() > 0 else "Tidak ada missing values")

print(f"\nShape before dropping NA: {df.shape}")
df = df.dropna()
print(f"Shape after dropping NA: {df.shape}")

missing_after = df.isnull().sum()
print("\nMissing values after drop:")
print(missing_after[missing_after > 0] if missing_after.sum() > 0 else "Tidak ada missing values")


Missing values per kolom:
Tahun                                                             30
Official Exchange Rate: Period Average: Local Currency to USD    180
Corruption Perception Index                                      180
GDP Growth(%)                                                    180
Current account balance (% of GDP)                               180
Inflation, consumer prices (annual %)                            180
Total Reserves in Month Imports                                  180
dtype: int64

Shape before dropping NA: (330, 8)
Shape after dropping NA: (150, 8)

Missing values after drop:
Tidak ada missing values


In [9]:
country_mapping = {
    'IDN': 'Indonesia', 'MYS': 'Malaysia', 'PHL': 'Philippines',
    'SGP': 'Singapore', 'THA': 'Thailand',
    'indonesia': 'Indonesia', 'malaysia': 'Malaysia',
    'philippines': 'Philippines', 'singapore': 'Singapore', 'thailand': 'Thailand',
    'Indonesia': 'Indonesia', 'Malaysia': 'Malaysia',
    'Philippines': 'Philippines', 'Singapore': 'Singapore', 'Thailand': 'Thailand'
}

if 'country' in df.columns:
    df['country'] = df['country'].replace(country_mapping)
    print(f"Unique countries: {df['country'].unique()}")


Unique countries: ['Indonesia' 'Malaysia' 'Filipina' 'Singapura' 'Thailand']


In [10]:
year_col = 'Tahun' if 'Tahun' in df.columns else 'year'
if year_col in df.columns:
    df = df.rename(columns={year_col: 'year'})
    df = df[(df['year'] >= START_YEAR) & (df['year'] <= END_YEAR)].copy()

coverage = df.groupby('country')['year'].agg(['min', 'max', 'count'])
print("Coverage per negara:")
print(coverage)


Coverage per negara:
              min     max  count
country                         
Filipina   1995.0  2024.0     30
Indonesia  1995.0  2024.0     30
Malaysia   1995.0  2024.0     30
Singapura  1995.0  2024.0     30
Thailand   1995.0  2024.0     30


In [11]:
df = df.sort_values(['country', 'year']).copy()

exchange_col = 'Official Exchange Rate: Period Average: Local Currency to USD'
if exchange_col in df.columns:
    df = df.rename(columns={exchange_col: 'exchange_rate'})
    df['log_exchange_rate'] = np.log(df['exchange_rate'])
    df['depreciation'] = df.groupby('country')['log_exchange_rate'].diff()
    print(f"Depreciation - Mean: {df['depreciation'].mean():.4f}, Std: {df['depreciation'].std():.4f}")
elif 'exchange_rate' in df.columns:
    df['log_exchange_rate'] = np.log(df['exchange_rate'])
    df['depreciation'] = df.groupby('country')['log_exchange_rate'].diff()
    print(f"Depreciation - Mean: {df['depreciation'].mean():.4f}, Std: {df['depreciation'].std():.4f}")


Depreciation - Mean: 0.0252, Std: 0.1296


In [12]:
reserves_col = 'Total Reserves in Month Imports'
if reserves_col in df.columns:
    df = df.rename(columns={reserves_col: 'reserves_moi'})

log_vars = ['reserves_moi']
for var in log_vars:
    if var in df.columns and (df[var] > 0).all():
        df[f'l{var}'] = np.log(df[var])
        print(f"Created l{var}")


Created lreserves_moi


In [13]:
cpi_col = 'Corruption Perception Index'
if cpi_col in df.columns:
    df['corr_inv'] = 100 - df[cpi_col]
    print("Created corr_inv (inverted CPI)")

gdp_growth_col = 'GDP Growth(%)'
if gdp_growth_col in df.columns:
    df = df.rename(columns={gdp_growth_col: 'gdp_growth'})

ca_gdp_col = 'Current account balance (% of GDP)'
if ca_gdp_col in df.columns:
    df = df.rename(columns={ca_gdp_col: 'ca_gdp'})

infl_col = 'Inflation, consumer prices (annual %)'
if infl_col in df.columns:
    df = df.rename(columns={infl_col: 'infl_diff'})

vars_to_lag = ['corr_inv', 'infl_diff', 'ca_gdp', 'reserves_moi', 'gdp_growth']
for var in vars_to_lag:
    if var in df.columns:
        df[f'{var}_l1'] = df.groupby('country')[var].shift(1)
        print(f"Created {var}_l1")


Created corr_inv (inverted CPI)
Created corr_inv_l1
Created infl_diff_l1
Created ca_gdp_l1
Created reserves_moi_l1
Created gdp_growth_l1


In [14]:
df['AFC_97_98'] = ((df['year'] >= 1997) & (df['year'] <= 1998)).astype(int)
df['GFC_08_09'] = ((df['year'] >= 2008) & (df['year'] <= 2009)).astype(int)
df['COVID_20_22'] = ((df['year'] >= 2020) & (df['year'] <= 2022)).astype(int)

print("Distribusi dummy krisis:")
for dummy in ['AFC_97_98', 'GFC_08_09', 'COVID_20_22']:
    print(f"{dummy}: {df[dummy].sum()} observasi")


Distribusi dummy krisis:
AFC_97_98: 10 observasi
GFC_08_09: 10 observasi
COVID_20_22: 15 observasi


In [15]:
vars_to_check = ['depreciation', 'corr_inv', 'infl_diff', 'ca_gdp', 'reserves_moi', 'gdp_growth']
outlier_report = {}

for var in vars_to_check:
    if var not in df.columns:
        continue
    data = df[var].dropna()
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[var] < lower_bound) | (df[var] > upper_bound)]
    if len(outliers) > 0:
        outlier_report[var] = len(outliers)
        print(f"{var}: {len(outliers)} outliers")

if len(outlier_report) == 0:
    print("Tidak ada outlier terdeteksi")


depreciation: 9 outliers
corr_inv: 30 outliers
infl_diff: 5 outliers
reserves_moi: 11 outliers
gdp_growth: 14 outliers


In [16]:
output_file = "Data/panel_asean5_cleaned.csv"
df.to_csv(output_file, index=False)
print(f"Data saved to: {output_file}")
print(f"Final shape: {df.shape}")
df.head()


Data saved to: Data/panel_asean5_cleaned.csv
Final shape: (150, 20)


,year,exchange_rate,Corruption Perception Index,gdp_growth,ca_gdp,infl_diff,reserves_moi,country,log_exchange_rate,depreciation,lreserves_moi,corr_inv,corr_inv_l1,infl_diff_l1,ca_gdp_l1,reserves_moi_l1,gdp_growth_l1,AFC_97_98,GFC_08_09,COVID_20_22
60,1995.0,25.6763,27.7,4.625225,-2.339200,6.831996,2.613809,Filipina,3.245568,NaN,0.960809,72.3,NaN,NaN,NaN,NaN,NaN,0,0,0
61,1996.0,26.2028,26.9,5.860348,-4.176524,7.476104,3.204902,Filipina,3.265866,0.020298,1.164681,73.1,72.3,6.831996,-2.339200,2.613809,4.625225,0,0,0
62,1997.0,29.5693,30.5,5.186412,-4.623494,5.590259,1.961715,Filipina,3.386737,0.120870,0.673819,69.5,73.1,7.476104,-4.176524,3.204902,5.860348,1,0,0
63,1998.0,40.7572,33.0,-0.514090,2.075379,9.234934,3.074316,Filipina,3.707633,0.320896,1.123082,67.0,69.5,5.590259,-4.623494,1.961715,5.186412,1,0,0
64,1999.0,39.0886,36.0,3.346451,-3.357069,5.939049,5.172271,Filipina,3.665831,-0.041802,1.643312,64.0,67.0,9.234934,2.075379,3.074316,-0.514090,0,0,0
